In [1]:
from nltk.corpus import wordnet as wn
import json
from itertools import chain

In [2]:
wordnetWords = set(chain(*[ss.lemma_names() for ss in wn.all_synsets()]))

In [8]:
audioLinks = {}
with open("kaikki.org-dictionary-English.json", "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        if 'sounds' in data:
            for j in data['sounds']:
                if ('mp3_url' in j or 'ogg_url' in j):
                    audioLinks[data['word']]= j['mp3_url']
                    continue 
               

In [50]:
def addPos(posDict,synset):
    posMap = {
        'n' : 'noun',
        'a' : 'adjective',
        'v' : 'verb',
        'r' : 'adverb'
    }
    pos = synset.pos()
    if pos == 's': return
    for i in posDict:
        if i['partOfSpeech'] == posMap[pos]:
            i['definitions'].append( {
                'definition' : synset.definition(),
                'example' :[synset.examples()[ :],"WordNet"] if synset.examples() else [],
            })
    posDict.append({
        'partOfSpeech' :posMap[pos],
        'definitions' : [
                            {
                                    'definition' : synset.definition(),
                                    'example' :[synset.examples()[ : ],"WordNet"] if synset.examples() else [],
                                }
        
        ]
    })        

        

In [51]:
finalDict = []
for word in wordnetWords:
    posDict = []
    for i in wn.synsets(word):
        addPos(posDict,i)
    currWordDict = {}
    currWordDict['word']=word
    currWordDict['usage'] = posDict
    currWordDict['source'] = "WordNet"
    currWordDict['audioLink'] = audioLinks[word] if word in audioLinks else ""
    finalDict.append(currWordDict)
    


In [52]:
finalDict = sorted(finalDict,key = lambda x:x['word'])
json_object = json.dumps(finalDict, indent=4)
with open("sample.json", "w") as outfile:
    outfile.write(json_object)        